## Task-01 Clue-02: Fetching S3 Bucket with dataset

In [ ]:
import boto3
  
s3_resource = boto3.resource('s3')
all_buckets = s3_resource.buckets.all()

for bucket in all_buckets:
    if (bucket.name.startswith('sagemaker')):
        BUCKET = bucket.name
        print ("SageMaker Default Bucket for Training Data: ", BUCKET)

jam_bucket = s3_resource.Bucket(BUCKET)

## Task-01 Clue-03: Extracting Zipped Dataset into S3

In [ ]:
import boto3
import zipfile
from io import BytesIO

ZIPPED_DATA = 'radiography_train_data.zip'
TRAIN_DATA_PREFIX = 'radiography_train_data'

print ("Unzipping ", ZIPPED_DATA)
zip_obj = s3_resource.Object(bucket_name=BUCKET, key=ZIPPED_DATA)
buffer = BytesIO(zip_obj.get()["Body"].read())
z = zipfile.ZipFile(buffer)
for filename in z.namelist():
    file_info = z.getinfo(filename)
    s3_resource.meta.client.upload_fileobj(
        z.open(filename),
        Bucket=BUCKET,
        Key=TRAIN_DATA_PREFIX+'/'+f'{filename}'
    )
print ("Completed Unzipping Training Data")

## Task-01 Clue-04: Extracting Zipped Dataset into S3

In [ ]:
import boto3

def get_size(bucket, path):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    total_size = 0
    image_count = 0
    for obj in my_bucket.objects.filter(Prefix=path):
        if ".png" in obj.key:
            total_size = total_size + obj.size
            image_count = image_count + 1
    return total_size, image_count

folder_size, image_count = get_size(BUCKET, PREFIX)
average_image_size = folder_size / image_count
print (average_image_size)